# Bernstein-Vazirani Algorithm

The Bernstein-Vazirani algorithm is a quantum algorithm that efficiently determines a secret bit string `s` encoded within an oracle.  Given a function (implemented as an oracle) that computes the bitwise dot product of an input `x` with the secret string `s`, modulo 2, the algorithm finds `s` with a *single* quantum query.  Classically, this would require *n* queries, where *n* is the length of the bit string.

**Key Concepts:**

*   **Oracle:** A "black box" function that we can query, but whose internal workings are unknown.  In this case, the oracle computes  `f(x) = s · x (mod 2)`.  This is the bitwise dot product of `x` and `s`, and then taking the result modulo 2 (i.e., the result is 0 or 1).
*   **Superposition:** The algorithm utilizes the superposition principle to query the oracle with all possible inputs simultaneously.
*   **Interference:**  Quantum interference is used to extract the secret string `s` from the superposition.

**We will:**

1.  Implement the Bernstein-Vazirani algorithm.
2.  Explain the role of each gate and step.
3.  Provide an example with a specific secret string.


## Step 1: Import Necessary Libraries

We'll use Qiskit for this implementation:

In [8]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator

We begin by importing the necessary libraries:

*   **Qiskit:**
    *   `QuantumCircuit`: For building quantum circuits.
    *   `QuantumRegister`: For creating groups of qubits.
    *   `ClassicalRegister`: For storing measurement results (classical bits).
    *   `transpile`: For optimizing circuits for a specific backend (simulator or hardware).
*   **Qiskit Aer:**
    *   `AerSimulator`: For simulating quantum circuits on a classical computer.

## Step 2: Define the Oracle Function

The oracle implements the function `f(x) = s · x (mod 2)`.  We'll create a function to generate the oracle circuit based on a given secret string `s`.

In [9]:
def create_oracle(secret_string):
    n = len(secret_string)
    oracle = QuantumCircuit(n + 1)

    # Reverse the secret string because Qiskit's qubit ordering is reversed
    reversed_string = secret_string[::-1]

    # Apply CNOT gates based on the secret string
    for i, bit in enumerate(reversed_string):
        if bit == '1':
            oracle.cx(i, n)  # CNOT with qubit i as control, last qubit as target

    return oracle

The `create_oracle` function generates the quantum circuit for the oracle:

*   **Function Definition:** `def create_oracle(secret_string):`
    *   `secret_string`: The secret bit string (e.g., "10110").

*   **Docstring:** Explains the function's purpose, arguments, and return value.

*   **Determine Number of Qubits:** `n = len(secret_string)`:  Gets the length of the secret string, which determines the number of qubits (plus one ancilla qubit).

*   **Create Circuit:** `oracle = QuantumCircuit(n + 1)`: Creates a `QuantumCircuit` with `n + 1` qubits.  The extra qubit (at index `n`) is the *ancilla* qubit, where the result of `f(x)` will be stored.

*   **Reverse String:** `reversed_string = secret_string[::-1]`:  Reverses the secret string. This is *crucial* because Qiskit uses a little-endian convention for qubit ordering (the least significant bit is qubit 0).  We need to reverse the string to align with this convention.

*   **Apply CNOT Gates:** `for i, bit in enumerate(reversed_string):`:
    *   Iterates through the *reversed* secret string, getting both the index (`i`) and the value (`bit`) of each bit.
    *   `if bit == '1':`: If the bit in the secret string is '1', apply a CNOT gate.
        *   `oracle.cx(i, n)`: Applies a CNOT gate.  Qubit `i` is the *control* qubit, and qubit `n` (the ancilla) is the *target* qubit.  This implements the dot product modulo 2. If the i-th bit of 's' is 1, then the CNOT gate flips the ancilla qubit *if and only if* the i-th input qubit is also 1.  This is equivalent to adding (modulo 2) the i-th bit of `s` multiplied by the i-th bit of `x` to the ancilla.

*   **Return Value:** `return oracle`: Returns the constructed oracle circuit.

**Key Idea of the Oracle:** The oracle effectively encodes the secret string `s` into a series of CNOT gates. Each '1' bit in `s` corresponds to a CNOT gate that entangles the corresponding input qubit with the ancilla qubit.

## Step 3: Define the Bernstein-Vazirani Algorithm Function

Now, we'll create the main function that implements the algorithm.

In [10]:
def bernstein_vazirani(oracle):

    n = oracle.num_qubits - 1  # Get the number of qubits (excluding ancilla)
    qreg = QuantumRegister(n + 1, 'q')
    creg = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(qreg, creg)

    # Initialize the ancilla qubit to |1>
    qc.x(qreg[n])

    # Apply Hadamard gates to all qubits
    qc.h(qreg)

    # Apply the oracle
    qc.compose(oracle, inplace=True)

    # Apply Hadamard gates to the input qubits again
    qc.h(qreg[:n])

    # Measure the input qubits
    qc.measure(qreg[:n], creg)

    return qc

The `bernstein_vazirani` function puts the algorithm together:

*   **Function Definition:** `def bernstein_vazirani(oracle):`
    *   `oracle`: The quantum circuit representing the oracle (created by `create_oracle`).

*   **Docstring:** Explains the function's purpose, arguments, and return value.

*   **Get Number of Qubits:** `n = oracle.num_qubits - 1`:  Determines the number of qubits used for the input (`n`) by subtracting 1 (the ancilla) from the total number of qubits in the oracle.

*   **Create Registers:**
    *   `qreg = QuantumRegister(n + 1, 'q')`: Creates a quantum register with `n + 1` qubits.
    *   `creg = ClassicalRegister(n, 'c')`: Creates a classical register with `n` bits (to store the measurement results).

*   **Create Circuit:** `qc = QuantumCircuit(qreg, creg)`: Creates the main quantum circuit.

*   **Initialize Ancilla:** `qc.x(qreg[n])`:  Applies an X gate to the ancilla qubit (qubit `n`). This puts the ancilla in the |1⟩ state. This is *essential* for the oracle to work correctly, creating the necessary phase kickback.

*   **Apply Hadamard Gates (Superposition):** `qc.h(qreg)`: Applies Hadamard gates to *all* qubits (including the ancilla). This creates a uniform superposition of all possible input states.

*   **Apply Oracle:** `qc.compose(oracle, inplace=True)`:  Combines the oracle circuit with the main circuit.

*   **Apply Hadamard Gates Again:** `qc.h(qreg[:n])`: Applies Hadamard gates to the *input* qubits (all qubits *except* the ancilla) again. This step creates the interference that reveals the secret string.

*   **Measure Input Qubits:** `qc.measure(qreg[:n], creg)`: Measures the input qubits (all qubits *except* the ancilla) and stores the results in the classical register.

*   **Return Value:** `return qc`: Returns the complete Bernstein-Vazirani circuit.

**How it Works (Intuitively):**

1.  **Superposition:** The initial Hadamard gates create a superposition of all possible inputs `x`.
2.  **Oracle Query:** The oracle applies a phase of (-1) to the states where `f(x) = s · x (mod 2) = 1`. This is a "phase kickback" effect due to the ancilla being initialized to |1⟩ and the CNOT gates in the oracle.
3.  **Interference:** The final Hadamard gates on the input qubits cause interference.  The amplitudes of the states constructively interfere for the state corresponding to the secret string `s` and destructively interfere for all other states.
4.  **Measurement:** Measuring the input qubits then yields the secret string `s` with certainty.


## Step 4: Example Usage

Let's demonstrate the algorithm with a specific secret string.

In [11]:
# Example Usage
secret_string = '101101'  # Example secret string
oracle = create_oracle(secret_string)
bv_circuit = bernstein_vazirani(oracle)

# Simulate the circuit
simulator = AerSimulator()
compiled_circuit = transpile(bv_circuit, simulator)
job = simulator.run(compiled_circuit, shots=1)
result = job.result()
counts = result.get_counts(bv_circuit)

print(f"Secret string: {secret_string}")
print(f"Measured result: {list(counts.keys())[0]}") # The key is the measured string
print(bv_circuit.draw())

Secret string: 101101
Measured result: 101101
     ┌───┐          ┌───┐             ┌─┐                   
q_0: ┤ H ├───────■──┤ H ├─────────────┤M├───────────────────
     ├───┤┌───┐  │  └┬─┬┘             └╥┘                   
q_1: ┤ H ├┤ H ├──┼───┤M├───────────────╫────────────────────
     ├───┤└───┘  │   └╥┘         ┌───┐ ║      ┌─┐           
q_2: ┤ H ├───────┼────╫───────■──┤ H ├─╫──────┤M├───────────
     ├───┤       │    ║       │  └───┘ ║ ┌───┐└╥┘     ┌─┐   
q_3: ┤ H ├───────┼────╫───────┼────■───╫─┤ H ├─╫──────┤M├───
     ├───┤┌───┐  │    ║  ┌─┐  │    │   ║ └───┘ ║      └╥┘   
q_4: ┤ H ├┤ H ├──┼────╫──┤M├──┼────┼───╫───────╫───────╫────
     ├───┤└───┘  │    ║  └╥┘  │    │   ║       ║ ┌───┐ ║ ┌─┐
q_5: ┤ H ├───────┼────╫───╫───┼────┼───╫───■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐  ║   ║ ┌─┴─┐┌─┴─┐ ║ ┌─┴─┐ ║ └───┘ ║ └╥┘
q_6: ┤ X ├┤ H ├┤ X ├──╫───╫─┤ X ├┤ X ├─╫─┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘  ║   ║ └───┘└───┘ ║ └───┘ ║       ║  ║ 
c: 6/═════════════════╩═══╩════════════

* **Define secret string**
*   **Create Oracle:** `oracle = create_oracle(secret_string)`: Creates the oracle circuit for the chosen secret string.

*   **Build Algorithm Circuit:** `bv_circuit = bernstein_vazirani(oracle)`: Creates the complete Bernstein-Vazirani circuit.

*   **Simulation:**
    *   `simulator = AerSimulator()`: Sets up the Qiskit Aer simulator.
    *   `compiled_circuit = transpile(bv_circuit, simulator)`: Optimizes the circuit for the simulator.
    *   `job = simulator.run(compiled_circuit, shots=1)`: Runs the simulation.  We use `shots=1` because the algorithm is deterministic (it always gives the correct answer with one measurement).
    *   `result = job.result()`: Gets the simulation results.
    *   `counts = result.get_counts(bv_circuit)`: Gets the measurement counts.

*   **Output:**
    *   `print(...)`: Prints the original secret string and the measured result.  They should match!
* **Drawing**: The cell also contains circuit's drawing